In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
import math
import os
import datetime
from pandas import DataFrame
import itertools
from bs4 import BeautifulSoup
import urllib.request
import requests
import random
import torch
import xmltodict
import xml.etree.ElementTree as ET
import xml
from xml.etree.ElementTree import parse
from xml.parsers import expat
from xml.dom.minidom import parse, parseString
import psycopg2
import PyPDF2
import pikepdf
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
import spacy

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
def createfiledirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    return directory

# **LOAD & TRANSFORM BILL XML DATA FROM LOCAL FILE**

In [3]:
def findkeywordlabel(headers):
    uniquewordpd=pd.DataFrame(columns=['WordText','WordTextLower', 'CategoryLabel']);
    nlp = spacy.load("en_core_web_sm");
    wordlabelpd=pd.DataFrame(columns=['BillID', 'HeaderID', 'HeaderOrderID',  'EntityOrderID', 'Text', 'CategoryLabel']);
    billidlist=[];
    for ind in headers.index:
        doc = nlp(str(headers.loc[ind,'Text']).replace('-', ' ').replace('     ', ''));
        entorder=0;
        for ent in doc.ents:
            entorder=entorder+1;
            if ent.text.find('Act')!=-1 or ent.text.find('ACT')!=-1:
                label='ACT';
            else:
                label=ent.label_;
            wordlabelpd=wordlabelpd.append({'BillID':headers.loc[ind, 'BillID'], 'HeaderID':headers.loc[ind, 'HeaderID'],'HeaderOrderID':headers.loc[ind, 'HeaderOrderID'],
                                            'EntityOrderID':entorder , 'HeaderText': ent.text, 'CategoryLabel': label}, ignore_index=True);
            if ent.text.lower() not in uniquewordpd['WordTextLower'].values:
                uniquewordpd=uniquewordpd.append({'WordText': ent.text, 'WordTextLower': ent.text.lower(), 'CategoryLabel':label}, ignore_index=True);
        if str(headers.loc[ind, 'BillID'])[-1]=='0' and headers.loc[ind, 'BillID'] not in billidlist:
            billidlist.append(headers.loc[ind, 'BillID']);
            print(headers.loc[ind, 'BillID']);
    return wordlabelpd, uniquewordpd

# **EXECUTE DATA IMPORT & TRANSFORM FUNCTIONS**

In [4]:
username='joythompson'
projfol="/Users/"+username+"/Desktop/DSE_203_JoyT/DSE_203_Final_Project_JoyT/"
directorybillhead=createfiledirectory(projfol+ 'Bill_Parsing_Headers/')
headerpd=pd.read_csv(directorybillhead+ 'header_text_by_bill_header_with_order.csv')

In [5]:
noungroupspd, uniquenounspd=findkeywordlabel(headerpd)

10
30
70
80
120
150
160
170
190
200
210
230
240
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590


In [6]:
noungroupspd.to_csv(directorybillhead+ 'nouns_by_bill_header_with_groups.csv')
uniquenounspd.to_csv(directorybillhead+ 'unique_nouns_with_groups.csv')